In [ ]:
import mechanicalsoup
import bs4, re, lxml, unidecode
import base64,glob,os
from appscript import *
import cellbell

In [ ]:
browser = mechanicalsoup.Browser()

In [ ]:
download='L1VzZXJzL2dpbGxlcy9Eb3dubG9hZHMv'
vrac='L1ZvbHVtZXMvZ2lsbGVzL1RyYW5zZmVydC9WcmFjMi8='
linkFilter=["c2l0ZXJpcA==","bGVzYmlhbg==","MTlbNzg5XVxk","c2hlbWFsZQ==","dHJhbnM=",'W3NTXXF1aXJ0']
urlBase='aHR0cDovL2Z1bGx4eHhtb3ZpZXMubmV0L3BhZ2UvJWQv'
numLettres="ABCDEFGHIJKLMNOP"

In [ ]:
numPage=1
nbPages=4
extension=""
safariLoad=False
boolPrint=False

In [ ]:
def en(s):
    return base64.b64encode(s)
def de(s):
    return base64.b64decode(s)
de("")

In [ ]:
def getSoup(url):
    page = browser.get(url)
    soup = bs4.BeautifulSoup(page.content,"lxml")
    return soup

In [ ]:
def getSubLinks(soup,tag="article"):
    links=[]
    for article in soup.find_all(tag):
        href=article.find('a')["href"]
        if linkFilter:
            lF=[de(l) for l in linkFilter]
            match="("+"|".join(lF)+")"
            m=re.search(match,href)
            if not m:
                links.append(article.find('a')["href"])
        else:
            links.append(article.find('a')["href"])
    return links

In [ ]:
def filterASoup(soup,chaine,splitter,link):
    for a in soup.find_all('a'):
        href=a.get("href")
        if href and chaine in href:
            link=href.split(splitter)[-1]
    return link

def getTwitterSecurelyLink(url):
    link=url
    soup=getSoup(url)
    link=filterASoup(soup,"url=https://openload.co","url=",link)
    if link==url:
        link=filterASoup(soup,"u=https://openload.co","u=",link)
    if link==url:
        print "no share link"
    return link

In [ ]:
def getSecurelyLink(url):
    link=url
    for meta in getSoup(url).find_all("meta"):
        content=meta.get("content")
        if content and "://openload.co" in content:
            link=content
    if link==url:
        print "no content link",link
        link=getTwitterSecurelyLink(url)
    elif "securely.link" in link:
        print "recursive securely",link
        link=getSecurelyLink(link)
    return link

In [ ]:
def getOpenloadLinks(soup):
    links=[]
    for article in soup.find_all("div",class_="entry-content"):
        aSoup=article.find_all('a')
        for a in aSoup:
            link=a["href"]
            if u"openload.co" in link:
                links.append(link)
            elif u"openload.co" in a.text.lower() and u"securely.link" in link:
                links.append(getSecurelyLink(link))
    return links

In [ ]:
def getNextPage(soup):
    try:
        nextPage=soup.find("a", class_="next page-numbers")["href"]
    #    print nextPage.rsplit("/")[-1]
        return getSoup(nextPage)
    except:
        return None

In [ ]:
def pageLoop(urls,safariLoad=True,boolPrint=False,missing=[]):
    olURLs={}
    for url in urls:
        urlSoup=getSoup(url)
        for nLink,link in enumerate(getOpenloadLinks(urlSoup)):
            if u"openload.co" in link:
                sLink=link.rsplit("/")[-1]
            else:
                sLink=link
            if not sLink[-4:]==".mp4":
                sLink+=".mp4"
            boolMissing=sLink in missing
            if (boolPrint or safariLoad) and not boolMissing:
                print urlSoup.title.text
                print sLink
                print link
            fileNames[sLink]=urlSoup.title.text+"-%d"%nLink
            olURLs[sLink]=link
            if safariLoad and not boolMissing:
                ok=raw_input("Press Enter to continue...")
                if ok.lower()!="n":
                    safari.windows.first.make(new=k.tab,with_properties={k.URL:link})
                else:
                    break
            elif boolMissing:
                safari.windows.first.make(new=k.tab,with_properties={k.URL:link})
    return olURLs

In [ ]:
def dictinvert(d):
    inv = {}
    for k, v in d.iteritems():
        keys = inv.setdefault(v, [])
        keys.append(k)
    return inv

In [ ]:
def normalizeName(name):
    result="".join([l.capitalize() for l in name.rsplit("-",1)[0].rsplit("(",1)[0].split(" ") if l!=""])
    result=result.replace("-","").replace("!","").replace("'","").replace(":","")
    return unidecode.unidecode(result)

In [ ]:
def suffixName(element):
    nameKey,suffix=fileNames[element].rsplit("-",1)
    suffix=int(suffix)
    name=normalizeName(fileNames[element])
    if len(nameFiles[nameKey])>1:
        name+=numLettres[suffix]    
    return name

In [ ]:
def stripName(name):
    return unidecode.unidecode(name.rsplit("-",1)[0])

In [ ]:
def getRepFiles(repName=de(download)):
    return [l.rsplit("/")[-1] for l in glob.glob(repName+u"*.mp4")]

In [ ]:
def getPages():
    olURLs={}
    localPage=page
    print linkPage.rsplit("/")[-1]
    for i in range(nbPages):
        print "page %02d"%i
        urls=getSubLinks(localPage)
        olURLs.update(pageLoop(urls,safariLoad=safariLoad,boolPrint=boolPrint,missing=missing))
        localPage = getNextPage(localPage)
        if not localPage:
            break
    return olURLs

In [ ]:
def getSelectNums(selectNums):
    missingNums=[]
    for num in selectNums:
        key=sorted(nameFiles.keys())[num]
        print key
        missingNums.extend(nameFiles[key])
    print missingNums
    missingURLs=[olURLs[name] for name in missingNums]
    return missingURLs

In [ ]:
def getSelection(selection):
    for link in selection:
        safari.windows.first.make(new=k.tab,with_properties={k.URL:link})

In [ ]:
safari = app("Safari")
repName=de(download)
fileNames={}
#nameFiles={}
try:
    missing
except NameError:
    missing = []
missing

In [ ]:
numPage=1
nbPages=4
extension=u""
linkPage=de(urlBase)%numPage
page=getSoup(linkPage+extension)
urls=getSubLinks(page)

In [ ]:
print getSoup(urls[0]).find("title").text
olURLs=getPages()

nameFiles=dictinvert({f:stripName(fileNames[f]) for f in fileNames})
repFiles=getRepFiles(de(vrac))+getRepFiles()
for num,name in enumerate(sorted(nameFiles.keys())):
    sub=normalizeName(name)
    if filter(lambda x: sub in x, repFiles):
        print "\t%02d+"%num,name
    else:
        print "%02d "%num,name


In [ ]:
strSelect=raw_input("Selected numbers\n")
if strSelect:
    selectNums=strSelect.split()
    selectNums=[int(n) for n in selectNums]
    print selectNums
    missing=getSelectNums(selectNums)
print len(missing)
strGetPages=raw_input("Get pages\n")
if strGetPages=="y":
    getSelection(missing)

In [ ]:
olURLs

In [ ]:
missing=[]

In [ ]:
getPages()

In [ ]:
fileNames

In [ ]:
repFiles=getRepFiles()
for element in sorted(repFiles):
    if element in fileNames:
        name=suffixName(element)
        print element, name
        os.rename(repName+element,repName+name+".mp4")

In [ ]:
def missedDownloads(missing=missing):
    missed=set()
    targets=set()

    for element in glob.glob(de(download)+"*.mp4*"):
        targets.add(element.split(".")[0].split("/")[-1])

    for element in missing:
        elementMain=element.split(".")[0]
        if not elementMain in targets:
            missed.add(element)
    missed=list(missed)
    return missed

In [ ]:
missedDownloads(missing)

In [ ]:
def longMissing(missing=missing):
    missingLong=set()
    for fileName in fileNames:
        for element in missing:
            if element in fileName:
                missingLong.add(fileName)
    return list(missingLong)

In [ ]:
missing=missedDownloads(missing)

In [ ]:
for element in missing:
    print fileNames[element]

In [ ]:
missingElements=set()
for element in missing:
    missingElements.add(fileNames[element])
print len(missingElements)

In [ ]:
en("")